# Классы

In [148]:
import numpy as np

In [149]:
class ZeroCrossingSignal:
    
    @property
    def amplitudes(self): return self.__amplitudes

    @property
    def phases(self): return self.__phases

    @property
    def biases(self): return self.__biases

    def __init__(self, signal, harmonic_hf, start_offset, start_period):
        offset = start_offset
        period = start_period

        amplitudes = []
        phases = []
        biases = []

        iteration = 0
        while offset < (len(signal) - np.floor(period)-1):

            phase = 2 * np.pi / period * harmonic_hf # связано со временеи обората, длинна сгустка - bunch ## похоже на фазу 

            harmonics_vector = np.zeros(3) # матрица гармоник от 0 до period 
            harmonics_vector[0] = np.sum(signal[offset:offset + int(period) + 1] * np.cos(phase * np.arange(int(period) + 1)))
            harmonics_vector[1] = np.sum(signal[offset:offset + int(period) + 1] * np.sin(phase * np.arange(int(period) + 1)))
            harmonics_vector[2] = np.sum(signal[offset:offset + int(period) + 1])

            P = self.find_P_b_matrix(phase, int(period))
            A = np.linalg.inv(P) @ harmonics_vector.T # ----

            amplitudes.append(np.sqrt(A[0]**2 + A[1]**2)) # амплитуда 
            phases.append(offset + (1 / phase * np.arctan2(A[0], -A[1]))) # фаза 
            biases.append(A[2]) # ----

            offset = int(phases[iteration] + period) # перечет офсета 

            period = 1 / 10 * (phases[iteration] - phases[iteration-10]) if iteration > 12 else period # перечсет периода для окна 

            iteration += 1

        self.__amplitudes = np.array(amplitudes)
        self.__phases = np.array(phases)
        self.__biases = np.array(biases)


    @staticmethod
    def find_P_b_matrix(phase: float, period: int):
        P_b = np.zeros((3, 3))
        cos_vals = np.cos(phase * np.arange(int(period) + 1))
        sin_vals = np.sin(phase * np.arange(int(period) + 1))

        P_b[0, 0] = np.sum(cos_vals**2)
        P_b[0, 1] = np.sum(cos_vals * sin_vals)
        P_b[0, 2] = np.sum(cos_vals)
        P_b[1, 0] = P_b[0, 1]
        P_b[1, 1] = np.sum(sin_vals**2)
        P_b[1, 2] = np.sum(sin_vals)
        P_b[2, 0] = P_b[0, 2]
        P_b[2, 1] = P_b[1, 2]
        P_b[2, 2] = int(period) + 1

        return P_b

In [150]:
class LorentzFactors:
    def __init__(self, phases, booster_perimeter, delta_time, speed_of_light=2.997925e10, proton_rest_mass = 938.256e6):
        self.__phase_difference = np.diff(phases)
        self.__beta = booster_perimeter / (speed_of_light * delta_time * self.phase_difference)
        self.__gamma = 1 / np.sqrt(1 - self.beta**2)
        self.__energy = (self.gamma-1) * proton_rest_mass

    @property
    def phase_difference(self): return self.__phase_difference

    @property
    def beta(self): return self.__beta

    @property
    def gamma(self): return self.__gamma

    @property
    def energy(self): return self.__energy

In [151]:
class ParametersFCTRF:
    def __init__(self, signal_fct, signal_rf, period, offset_fct, offset_rf, offset_between_signals,
                 harmonic_hf, booster_perimeter, delta_time, averaging_window_for_finding_fct_minimums,
                 averaging_window_for_phases, phase_offset=0, charge_number = 28, proton_rest_mass = 938.256e6,
                 electron_charge = 1.6021e-19, speed_of_light = 2.997925e10):
        self.__zero_crossing_rf = ZeroCrossingSignal(signal_rf, harmonic_hf, offset_rf, period)
        self.__phases = self.__zero_crossing_rf.phases
        self.__lorentz_factors_rf = LorentzFactors(self.zero_crossing_rf.phases, booster_perimeter, delta_time, speed_of_light, proton_rest_mass)
        self.__discretized_phases = np.floor(self.zero_crossing_rf.phases[1:]).astype(int)
        self.__discretized_phases_with_offset = self.discretized_phases[phase_offset:]
        self.__fct_minimums = self.signal_minimums_with_averaging(signal_fct[offset_between_signals:], self.discretized_phases_with_offset,
                                                                averaging_window_for_finding_fct_minimums)
        self.__corrected_phases = self.correct_phases(signal_fct[offset_between_signals:], self.fct_minimums, self.zero_crossing_rf.phases,
                                                      phase_offset, harmonic_hf)
        self.__averaged_phases = np.convolve(np.hstack((np.zeros(phase_offset), self.corrected_phases)), #Зачем-то добавляется два нуля...
                                    np.ones(averaging_window_for_phases)/averaging_window_for_phases,
                                    mode='valid')[::averaging_window_for_phases][:-1] #Зачем-то откидывается последнее...

        self.__intensities = self.find_intensities(signal_fct[offset_between_signals:], self.fct_minimums, self.discretized_phases_with_offset,
                                        delta_time, charge_number=charge_number, electron_charge=electron_charge)       
    
    @property
    def phases(self): return self.__phases

    @property
    def zero_crossing_rf(self): return self.__zero_crossing_rf

    @property
    def lorentz_factors_rf(self): return self.__lorentz_factors_rf

    @property
    def discretized_phases(self): return self.__discretized_phases

    @property
    def discretized_phases_with_offset(self): return self.__discretized_phases_with_offset

    @property
    def fct_minimums(self): return self.__fct_minimums

    @property
    def corrected_phases(self): return self.__corrected_phases

    @property
    def averaged_phases(self): return self.__averaged_phases

    @property
    def intensities(self): return self.__intensities

    @staticmethod
    def correct_phases(signal, signal_minimums, phases, phase_offset, harmonic_hf):
        discretized_phases = np.int32(phases[1:])
        corrected_phases = np.zeros(len(discretized_phases) - phase_offset - 1, dtype=float)

        for i in range(len(corrected_phases)):
            up_sum = np.sum((signal[discretized_phases[phase_offset+i]: discretized_phases[phase_offset+i+1]+1] - signal_minimums[i])*\
                            (np.arange(discretized_phases[phase_offset+i+1]-discretized_phases[phase_offset+i]+1) + discretized_phases[phase_offset+i] -\
                               (phases[phase_offset+i+2] + phases[phase_offset+i+1]) / 2))
            under_sum = np.sum(signal[discretized_phases[phase_offset+i]:
                                    discretized_phases[phase_offset+i+1]+1] - signal_minimums[i])
            corrected_phases[i] = 360 * harmonic_hf / (discretized_phases[phase_offset+i+1] - discretized_phases[phase_offset+i]) * (up_sum / under_sum)
        
        return corrected_phases

    @staticmethod
    def find_intensities(signal, signal_minimums, phases, delta_time, charge_number = 28, electron_charge=1.6021e-19):
        return np.fromiter((8.6 * 10**(-5) * (10**(-3) / (5 * charge_number)) *\
            (delta_time / electron_charge) * np.sum(signal[phases[i]: phases[i+1] + 1] - signal_minimums[i])
            for i in range(len(phases)-1)), float) / 10 ## НЕ ДО КОНЦА ПОНЯЛ, ПОЧЕМУ НАДО ДЕЛИТЬ НА ДЕСЯТЬ, МБ ОШИБКА В ДРУГОМ МЕСТЕ 

    @staticmethod
    def signal_minimums_with_averaging(signal, period_nodes, window_length):
        result = np.zeros(len(period_nodes)-1)
        for i in range(len(period_nodes)-1):
            result[i] = np.convolve(signal[period_nodes[i]: period_nodes[i+1]], np.ones(window_length) / window_length, mode='valid').min()
        return result

# Константы

In [152]:
mass_proton = 938.256 * 10**6 # масса покоя протона 
charge_electron  = 1.6021 * 10**(-19) # заряд электрона в Кл
speed_of_light = 2.997925 * 10**10

# Параметры

In [153]:
booster_perimeter = 21096 # периметр бустера в мм #ВЕЗДЕ ОДИН
delta_time = (50 * 10e5)**(-1) #ВЕЗДЕ ОДНО

# оффсеты по какой-то причине отличаются у некоторых, это нужно уточнить
offset_fct = 45 #нафиг не нужен в классе

#ОБЩИЕ ПАРАМЕТРЫ
booster_perimeter = 21096 # периметр бустера в мм
delta_time = (5e6)**(-1) 
offset_between_signals = 10
data_length = 7 * 32 * 128 * 1024 + 1
phase_offset = 2
averaging_window_for_finding_fct_minimums = 13
averaging_window_for_phases = 20
last_window_first_record = 68_000_000

In [154]:
keys = ['1_start', '1_end', '2_start', '2_end']

period = {'1_start': 425.067, '1_end': 127.848, '2_start': 425.067, '2_end': 425.067}
harmonic_hf = {'1_start': 5, '1_end': 1, '2_start': 5, '2_end': 1}
offset_rf = {'1_start': 10, '1_end': 112, '2_start': 10, '2_end': 112}

signal_fct = {'1_start': np.fromfile('data/booster_acceleration/1/27_01_23_booster_fct_1.bin', dtype='int16')[:data_length],
              '1_end': np.fromfile('data/booster_acceleration/1/27_01_23_booster_fct_1.bin', dtype='int16', offset=last_window_first_record)[:data_length],
              '2_start': np.fromfile('data/booster_acceleration/2/27_01_23_booster_fct_2.bin', dtype='int16')[:data_length],
              '2_end': np.fromfile('data/booster_acceleration/2/27_01_23_booster_fct_2.bin', dtype='int16', offset=last_window_first_record)[:data_length],}

signal_rf= {'1_start': np.fromfile('data/booster_acceleration/1/27_01_23_booster_rf_1.bin', dtype='int16')[:data_length],
              '1_end': np.fromfile('data/booster_acceleration/1/27_01_23_booster_rf_1.bin', dtype='int16', offset=last_window_first_record)[:data_length],
              '2_start': np.fromfile('data/booster_acceleration/2/27_01_23_booster_rf_2.bin', dtype='int16')[:data_length],
              '2_end': np.fromfile('data/booster_acceleration/2/27_01_23_booster_rf_2.bin', dtype='int16', offset=last_window_first_record)[:data_length],}

In [155]:
parameters_fct_rf = {}
for key in keys:
    parameters_fct_rf[key] = ParametersFCTRF(signal_fct=signal_fct[key], signal_rf=signal_rf[key], period=period[key],
                                             offset_fct=offset_fct, offset_rf=offset_rf[key], offset_between_signals=offset_between_signals,
                                             harmonic_hf=harmonic_hf[key], booster_perimeter=booster_perimeter, delta_time=delta_time,
                                             phase_offset=phase_offset, averaging_window_for_finding_fct_minimums=averaging_window_for_finding_fct_minimums,
                                             averaging_window_for_phases=averaging_window_for_phases)

In [191]:
# import pandas as pd

# parameters_from_dan = {}
# with open('Acceleration in Booster measured with FCT/BoosterAcceleration1_1half.dat', "rb") as file:
#     parameters_from_dan['1_start'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])
# with open('Acceleration in Booster measured with FCT/BoosterAcceleration1_2half.dat', "rb") as file:
#     parameters_from_dan['1_end'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])
# with open('Acceleration in Booster measured with FCT/BoosterAcceleration2_1half.dat', "rb") as file:
#     parameters_from_dan['2_start'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])
# with open('Acceleration in Booster measured with FCT/BoosterAcceleration2_2half.dat', "rb") as file:
#     parameters_from_dan['2_end'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])

In [192]:
parameters_from_dat = {}
with open('data/booster_acceleration/1/BoosterAcceleration1_1half.dat', "rb") as file:
    parameters_from_dat['1_start'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])
with open('data/booster_acceleration/1/BoosterAcceleration1_2half.dat', "rb") as file:
    parameters_from_dat['1_end'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])
with open('data/booster_acceleration/2/BoosterAcceleration2_1half.dat', "rb") as file:
    parameters_from_dat['2_start'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])
with open('data/booster_acceleration/2/BoosterAcceleration2_2half.dat', "rb") as file:
    parameters_from_dat['2_end'] = pd.DataFrame(np.load(file).reshape(-1,3), columns=['Phases', 'Intensities', 'Corrected phases'])

In [193]:
parameters_from_file = {}
for key in keys:
    parameters_from_file[key] = pd.DataFrame(
        {'Phases': parameters_fct_rf[key].phases[phase_offset: -phase_offset],
         'Intensities': parameters_fct_rf[key].intensities,
         'Corrected phases': parameters_fct_rf[key].corrected_phases})

In [194]:
for key in keys:
    print(key)
    print((parameters_from_dat[key] - parameters_from_file[key]).max(), '\n')

1_start
Phases                   0.033468
Intensities         227341.723878
Corrected phases         4.981631
dtype: float64 

1_end
Phases              0.000000e+00
Intensities         2.607703e-08
Corrected phases    1.705303e-13
dtype: float64 

2_start
Phases              0.000000e+00
Intensities         1.862645e-08
Corrected phases    4.263256e-13
dtype: float64 

2_end
Phases             -9.489286e+02
Intensities         1.856038e+06
Corrected phases    8.718868e+01
dtype: float64 

